In [1]:
%load_ext autoreload
%aimport -pandas, -numpy
%autoreload 2
import pandas as pd
import numpy as np
import arrow
from dfsm import msgFSM

In [2]:
import dmyplant2
dmyplant2.cred()
mp = dmyplant2.MyPlant(0)


In [3]:
dval = dmyplant2.Validation.load_def_csv("input2.csv")
i = 5
vale = dval.iloc[i]
#[['serialNumber','Validation Engine','val start']]
vale

n                                                    5
Validation Engine                           REGENSBURG
serialNumber                                   1243362
val start                          2020-09-07 00:00:00
oph@start                                        62765
starts@start                                       NaN
Asset ID                                      157701.0
Old PU first replaced OPH                       4913.0
Old PUs replaced before upgrade                    1.0
Name: 5, dtype: object

In [4]:
sn = 1243362
e=dmyplant2.Engine.from_sn(mp,sn, valstart='2020-09-07')
pd.DataFrame.from_dict(e.dash, orient='index').T

,Name,Engine ID,Design Number,Engine Type,Engine Version,P,P_nom,BMEP,serialNumber,id,Count_OpHour,val start,oph@start,oph parts,LOC
0,BMW Regensburg,M2,I448,616,J01,16,2744.9,22.0,1243362,157701,71738.0,2020-09-07,0,71738,None


In [5]:
res = e.get_keyItem_data('dataItems','Count_OpHour')
{ res['id'] : [res['name'],res['unit']] }

{161: ['Count_OpHour', 'h']}

In [6]:
e.get_keyItem('Count_OpHour')

{'id': 161,
 'name': 'Count_OpHour',
 'timestamp': 1645284869782,
 'unit': 'h',
 'value': 71784.0}

In [7]:
e.get_keyId('Count_OpHour')

161

In [8]:
datal = ['Count_OpHour','Power_PowerAct','Hyd_PressCrankCase','Hyd_PressOilDif']
dat = e.get_dataItems(['Count_OpHour','Power_PowerAct','Hyd_PressCrankCase','Hyd_PressOilDif'])

In [31]:
#def _batch_hist_dataItems(self, itemIds={161: ['CountOph', 'h']}, p_limit=None, p_from=None, p_to=None, timeCycle=3600,
#                              assetType='J-Engine', includeMinMax='false', forceDownSampling='false'):
e._batch_hist_dataItems(dat, p_from=1645281269000, p_to=1645281269000).iloc[0]

time                        1645281269000
Count_OpHour                 71782.425063
Power_PowerAct                1976.519094
Hyd_PressOilDif                  0.572489
Hyd_PressCrankCase             -15.344637
datetime              2022-02-19 14:34:29
Name: 0, dtype: object

In [80]:
def fetch_dataItems(e, items, ts):
    itemIds = e.get_dataItems(items)
    tdj = ','.join([str(s) for s in itemIds])
    url=fr"/asset/{e['id']}/history/batchdata?assetType=J-Engine&from={ts}&to={ts}&dataItemIds={tdj}&timeCycle=30"
    data =  e._mp.fetchdata(url)
    # restructure data to dict
    ds = {}
    ds['labels'] = ['timestamp'] + [itemIds[x][0] for x in data['columns'][1]]
    ds['data'] = [[r[0]] + [rr[0] for rr in r[1]] for r in [data['data'][0]]]
    # import to Pandas DataFrame
    df = pd.DataFrame(ds['data'], columns=ds['labels'])
    return df

fetch_dataItems(e, ['Count_OpHour','Power_PowerAct','Hyd_PressCrankCase','Hyd_PressOilDif'], 1645281269000)

,timestamp,Count_OpHour,Hyd_PressCrankCase,Hyd_PressOilDif,Power_PowerAct
0,1645281269000,71782.007533,-15.90832,0.572559,2009.9376


In [100]:
e.fetch_dataItems(1645281269000,['Count_OpHour','Various_Values_SpeedAct','Power_PowerAct','Power_SetPower','Hyd_PressCrankCase','Hyd_PressCoolWat','rP_Ramp_Set'])

,timestamp,rP_Ramp_Set,Hyd_PressCoolWat,Hyd_PressCrankCase,Power_PowerAct,Power_SetPower,Various_Values_SpeedAct,Count_OpHour
0,1645281269000,0.63,1.56,-15.90832,2009.9376,2006.641167,1499.0,71782.007533


In [10]:
e.asset['validation']

{'n': {'name': 'n', 'value': 0},
 'Validation Engine': {'name': 'Validation Engine', 'value': 'BMW Regensburg'},
 'serialNumber': {'name': 'serialNumber', 'value': 1243362},
 'val start': {'name': 'val start', 'value': Timestamp('2020-09-07 00:00:00')},
 'oph@start': {'name': 'oph@start', 'value': 0},
 'starts@start': {'name': 'starts@start', 'value': 0},
 'Asset ID': {'name': 'Asset ID', 'value': 157701},
 'Old PU first replaced OPH': {'name': 'Old PU first replaced OPH',
  'value': None},
 'Old PUs replaced before upgrade': {'name': 'Old PUs replaced before upgrade',
  'value': None},
 'RMD_ListBuffMAvgOilConsume_OilConsumption': {'value': None}}

In [13]:
e['Power_PowerNominal']

2679.0

In [12]:
#mp._fetch_installed_base()
e.lookup_Installed_Fleet(mp,e._sn)

{'Count_OpHour': 71738.0,
 'OperationalCondition': 'Running',
 'startup_counter': 186.0,
 'shutdown_counter': 185.0,
 'id': 157701,
 'Engine Version': 'J01',
 'Engine Type': '616',
 'Engine Series': '6',
 'Country': 'DE',
 'Commissioning Date': '2013-09-15',
 'Design Number': 'I448',
 'serialNumber': '1243362',
 'Power_PowerNominal': 2679.0,
 'Para_Speed_Nominal': 1500.0,
 'starts_oph_ratio': 0.024,
 'Control System Type': 'XT3',
 'Engine ID': 'M2',
 'IB Unit Commissioning Date': '2011-07-03',
 'IB NOX': '500',
 'IB Frequency': '50',
 'IB Item Description Engine': 'Repairs.ENG.JMS.616',
 'IB Site Name': 'BMW Regensburg',
 'IB Status': 'Active',
 'Contract.Warranty End Date': nan,
 'Contract.Warranty Start Date': nan,
 'Module_Vers_HalIO': '4.8.2.1.0',
 'Product Program': nan}

In [47]:

e.get_messages()

,index,assetId,isPrimary,message,name,severity,timestamp,associatedValues
0,47845,117227,NaN,Collective message: Software module status rep...,1991,700,1536765826555,NaN
1,47844,117227,NaN,Collective message: Software module reports wr...,1999,700,1536765826681,NaN
2,47843,117227,NaN,Collective message: Software module status rep...,1992,700,1536765826685,NaN
3,47842,117227,NaN,Service selector switch Off,1225,600,1536765829148,NaN
4,47841,117227,NaN,Ready for automatic off,1230,600,1536765829155,NaN
...,...,...,...,...,...,...,...,...
47841,4,117227,False,Hourly message,9007,600,1644989830478,NaN
47842,3,117227,False,Hourly message,9007,600,1644993430818,NaN
47843,2,117227,False,Hourly message,9007,600,1644997030656,NaN
47844,1,117227,False,Hourly message,9007,600,1645000630551,NaN


In [14]:
for k in sorted(e.properties.keys()):
    print(k)

Application Type
AssetStatusOracle
CSA Region
Catalyst Model
Commissioning Date
Contract Organization Unit
Contract.Service Contract End Counter
Contract.Service Contract End Date
Contract.Service Contract Start Counter
Contract.Service Contract Start Date
Contract.Service Contract Type
Control System Type
Country
Customer Engine Number
Design Number
End Customer
Engine ID
Engine Series
Engine Type
Engine Version
EngineSerialNumberOracle
Exhaust After-Treatment System Model
Gas Type
Gas Type2
Generator Model
Generator Serial Number
IB Alternator Item Number
IB Alternator Manufacturer
IB Cat2
IB City
IB Control Software
IB Customer Service Manager
IB Distribution Channel
IB Frequency
IB HQ Duns Number
IB Item Description Engine
IB ItemInstance Unit
IB ItemNumber Engine
IB ItemNumber Unit
IB Leanox
IB NOX
IB Operating Unit
IB Party Name Owner
IB Postal code
IB Project Name
IB Region
IB Service Region
IB Site Customer Name
IB Site Name
IB Status
IB Unit Commissioning Date
IB Unit Item Ins

In [25]:
100 / e['rP_Ramp_Set']

158.73015873015873

In [20]:
'Ramp' in 'rP_Ramp_Act'

True

In [16]:
for i, k in enumerate(sorted(e.properties.keys())):
    print(f"{i:03} {k}: {e[k]}")

000 Application Type: Industry
001 AssetStatusOracle: Active
002 CSA Region: DE
003 Catalyst Model: C
004 Commissioning Date: 2013-09-15
005 Contract Organization Unit: DE
006 Contract.Service Contract End Counter: 119999
007 Contract.Service Contract End Date: 2031-07-31
008 Contract.Service Contract Start Counter: 0
009 Contract.Service Contract Start Date: 2010-07-19
010 Contract.Service Contract Type: PREVENTIVE AND CORRECTIVE
011 Control System Type: XT3
012 Country: DE
013 Customer Engine Number: 2
014 Design Number: I448
015 End Customer: BMW AG
016 Engine ID: M2
017 Engine Series: 6
018 Engine Type: 616
019 Engine Version: J01
020 EngineSerialNumberOracle: JEN-1243362
021 Exhaust After-Treatment System Model: No
022 Gas Type: NG1
023 Gas Type2: n.a.
024 Generator Model: DIG 142 d/4
025 Generator Serial Number: 8430506A304
026 IB Alternator Item Number: 1203563
027 IB Alternator Manufacturer: AVK
028 IB Cat2: C
029 IB City: REGENSBURG
030 IB Control Software: V3.10.0
031 IB Cust

In [36]:
for i, k in enumerate(sorted(e.dataItems.keys())):
    if not k.startswith('Para_') and e[k] != None:
        #if k.startswith('Knock_KLS98_Knock'):
            print(f"{i:04} {k}: {e[k]}")

0002 Application Type: O&G Power Gen
0003 AssetStatusOracle: Active Docu incomplete
0007 Aux_GasRail_rPr_Out_Calc: -0.97
0009 Aux_GasRail_rPr_sp: 4.88
0013 Aux_PreChambDifPress: -514.0
0014 Aux_PreChambGasTrainPress: 4.5
0017 Aux_PreChambGasTrain_rPr_a_Filt: 5.7
0018 Aux_PreChambGasTrain_rPr_sp: 3.67
0019 Aux_RoomTemp: 21.9
0022 Aux_rPr_Baro: 1014.0
0023 BBFRULABS_RUL: 4706.0
0024 BMEP: 1.2438387691596453
0025 BaseProg_Vers: 4.4.0.8
0065 CMU_rDPr_Ch_AirFilt: -11.2
0066 CMU_rDPr_Ch_FA: 14.0
0067 CMU_rDPr_Ch_HPCooler: 51.0
0069 CMU_rPr_Ch_P2HP: 5.11
0071 CMU_rPr_Ch_P2bFA: 5.06
0072 CMU_rPr_a_BbFilt: -76.5
0075 CMU_rPr_f_BbFilt: -71.2
0078 Catalyst Model: --
0127 Contract Organization Unit: AT
0129 Cooling_Water_Degassing: 0.01
0130 Cooling_Water_TopUp: 2.01
0141 Count_Maintenance1: 174.0
0142 Count_Maintenance2: 7700.0
0143 Count_Maintenance3: 91983.0
0144 Count_Maintenance4: 5665.0
0145 Count_Maintenance5: 692.0
0146 Count_Maintenance6: 584096.0
0147 Count_Maintenance7: 684096.0
0148 Co

In [23]:
for k in sorted(e.dataItems.keys()):
    if 'Ramp' in k:
        print(k, e[k])

Para_Electrical_Grid_GridCodeStatic_FrequencyCtr_PowerRamp_Calc_PRamp_FC_active 1.0
Para_Electrical_Grid_GridCodeStatic_FrequencyCtr_PowerRamp_PRamp_FC_DecGrad_Norm 0.84
Para_Electrical_Grid_GridCodeStatic_FrequencyCtr_PowerRamp_PRamp_FC_ExtInp 1.0
Para_Electrical_Grid_GridCodeStatic_FrequencyCtr_PowerRamp_PRamp_FC_IncGrad_Norm 0.84
Para_Electrical_Grid_GridCodeStatic_FrequencyCtr_PowerRamp_PRamp_FC_SetCOn 1.0
Para_Electrical_Grid_GridCodeStatic_FrequencyCtr_PowerRamp_tiSetTime_FC_PRamp 30.0
Para_Engine_Island_Ramp_P_Island_Split_DecGrad_Norm1 0.6
Para_Engine_Island_Ramp_P_Island_Split_IncGrad_Norm1 0.6
Para_Engine_Leanox_LeanoxRamp 0.5
Para_GeneratCtr_Voltage_RampeVoltAdj 0.5
Para_Grid_GridCodeStatic_010_Ramp_ActPRampMainFailure None
Para_Grid_GridCodeStatic_010_Ramp_ActPRampUtility None
Para_Grid_GridCodeStatic_010_Ramp_ActSlowRampPFC None
Para_Grid_GridCodeStatic_010_Ramp_PRampMainFailure 0.1
Para_Grid_GridCodeStatic_010_Ramp_RampUtility 0.84
Para_Grid_GridCodeStatic_010_Ramp_TimeDe

In [37]:
for i, k in enumerate(sorted(e.dataItems.keys())):
    if not k.startswith('Para_') and e[k] != None:
        if k[-2:] in [f"{i:02}" for i in range(1,25)]:
            print(f"{i:04} {k}: {e[k]}")

0202 CylState_10: 2.0
0203 CylState_11: 2.0
0204 CylState_12: 2.0
0205 CylState_13: 2.0
0206 CylState_14: 2.0
0207 CylState_15: 2.0
0208 CylState_16: 2.0
0209 CylState_17: 2.0
0210 CylState_18: 2.0
0211 CylState_19: 2.0
0213 CylState_20: 2.0
0238 Exhaust_TempCyl01: 535.0
0239 Exhaust_TempCyl02: 528.0
0240 Exhaust_TempCyl03: 529.0
0241 Exhaust_TempCyl04: 538.0
0242 Exhaust_TempCyl05: 538.0
0243 Exhaust_TempCyl06: 535.0
0244 Exhaust_TempCyl07: 537.0
0245 Exhaust_TempCyl08: 529.0
0246 Exhaust_TempCyl09: 546.0
0247 Exhaust_TempCyl10: 546.0
0248 Exhaust_TempCyl11: 546.0
0249 Exhaust_TempCyl12: 549.0
0250 Exhaust_TempCyl13: 548.0
0251 Exhaust_TempCyl14: 547.0
0252 Exhaust_TempCyl15: 548.0
0253 Exhaust_TempCyl16: 544.0
0331 Ignition_ITPCyl01: 22.9
0332 Ignition_ITPCyl02: 22.9
0333 Ignition_ITPCyl03: 22.9
0334 Ignition_ITPCyl04: 22.9
0335 Ignition_ITPCyl05: 22.9
0336 Ignition_ITPCyl06: 22.9
0337 Ignition_ITPCyl07: 22.9
0338 Ignition_ITPCyl08: 22.9
0339 Ignition_ITPCyl09: 22.9
0340 Ignition_ITP